In [114]:
#!pip install plotguy
#!pip install hkfdb
#!pip install yfinance --upgrade
#!pip install talib-binary

In [115]:
import os
import copy
import datetime
import time
import itertools

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
#import plotguy

import yfinance as yf
import talib as ta

In [116]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Defining Data Folder Structures

In [117]:
#data folder structures
data_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/data'
backtest_output_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/backtest_output'

Defining Tickers List, Date Range and Data Retrival Intervals

In [118]:
# define list of tickers to backtest
ticker_list = ['ARCC', 'STAG', 'O', 'WPC', 'MAIN', 'ABR', 'ACRE', 'DIVO', 'JEPI', 'LTC', 'OHI', 'PFLT', 'QYLD', 'SCHD', 'STOR', 'XRMI']

# define date range
start_date = '2021-03-20'
end_date = '2022-11-10'
interval = '1d'

# initial capital in USD
initial_capital = 100000
last_realized_capital = copy.deepcopy(initial_capital)

**Getting OHLC Data from Yahoo Finance** </b>


*   Get stock data based on ticker list from Yahoo Finance
*   Create each stock data as different dataframe
*   Add Technical Analysis indicators from TA-Lib
*   Pointing working directory to data folder, export dach dataframe as independent csv file 





In [137]:
# changing working directory to data folder's path
data_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/data'
os.chdir(data_folder)

# getting OHLC data from yfinance package, if auto_adjust=True, OHLC data will not have adj close column, use progress=False to get rid of comments
# Other indicators: 14 day RSI
for ticker in ticker_list:
  try:
    globals()[ticker] = pd.DataFrame
    globals()[ticker] = yf.download(ticker, start=start_date, end=end_date, interval=interval, auto_adjust=True, back_adjust=True, progress=False)
    globals()[ticker]['200EMA'] = ta.EMA(globals()[ticker]['Close'], timeperiod=200) 
    globals()[ticker]['ATR'] = ta.ATR(globals()[ticker]['High'], globals()[ticker]['Low'], globals()[ticker]['Close'], timeperiod=14)
    globals()[ticker]['RSI'] = ta.RSI(globals()[ticker]['Close'], timeperiod=14)
    globals()[ticker]['MACD'], globals()[ticker]['MACD_signal'], globals()[ticker]['MACD_hist'] = ta.MACD(globals()[ticker]['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    globals()[ticker]['ADX'] = ta.ADX(globals()[ticker]['High'], globals()[ticker]['Open'], globals()[ticker]['Close'], 14)
    globals()[ticker]['ADXR'] = ta.ADXR(globals()[ticker]['High'], globals()[ticker]['Open'], globals()[ticker]['Close'], 14)
    globals()[ticker] = globals()[ticker].drop('Volume', axis=1)
    globals()[ticker] = globals()[ticker].round(4).dropna()
    
    #writing each ticker dataframe into separate .csv files, save to google drive
    #globals()[ticker].to_csv(ticker+'.csv', index=False, header=True)
  except:
    print(globals()[ticker])

In [120]:
df_list = [ARCC, STAG, O, WPC, MAIN, ABR, ACRE, DIVO, JEPI, LTC, OHI, PFLT, QYLD, SCHD, STOR, XRMI]

In [138]:
ARCC.head()

,Open,High,Low,Close,200EMA,ATR,RSI,MACD,MACD_signal,MACD_hist,ADX,ADXR
Date,,,,,,,,,,,,
2022-01-03,19.7110,19.7762,19.5341,19.5620,17.9626,0.2842,64.8029,0.1954,0.0806,0.1148,29.8459,30.0643
2022-01-04,19.6458,19.8879,19.5992,19.7668,17.9806,0.2871,68.4688,0.2228,0.1090,0.1138,30.6974,30.7831
2022-01-05,19.7575,19.8134,19.3851,19.4782,17.9955,0.2972,59.1241,0.2187,0.1310,0.0877,31.4880,31.3008
2022-01-06,19.4875,19.7296,19.3944,19.4968,18.0104,0.2999,59.5081,0.2145,0.1477,0.0668,30.8571,31.6315
2022-01-07,19.5061,19.7948,19.4968,19.7575,18.0278,0.2998,64.5313,0.2295,0.1640,0.0655,30.4583,32.2680


**Back Testing** </b>


*   Backtesting of Trade Logic across all tickers and in the designated timeframe
*   Backtesting for Trade strategy with simulated PNL



In [122]:
# Class to call Pandas by row and column
# https://stackoverflow.com/questions/28754603/indexing-pandas-data-frames-integer-rows-named-columns 
class XLocIndexer:
    def __init__(self, frame):
        self.frame = frame
    
    def __getitem__(self, key):
        row, col = key
        return self.frame.iloc[row][col]

pd.core.indexing.IndexingMixin.xloc = property(lambda frame: XLocIndexer(frame))

In [123]:
def backtest(df):
  pos_opened = False
  open_price  = 0
  close_price = 0
  pnl = 0
  pnl_list = []
  start_index = 1
  end_index = len(df)


  for i in range(start_index, end_index):
    now_open = df.xloc[i, 'Open']
    now_high = df.xloc[i, 'High']
    now_low  = df.xloc[i, 'Low']
    now_close = df.xloc[i, 'Close']
    now_200EMA = df.xloc[i, '200EMA']
    now_rsi   = df.xloc[i, 'RSI']
    now_MACD        = df.xloc[i, 'MACD']
    now_MACD_signal = df.xloc[i, 'MACD_signal']

    # opening a position
    if (pos_opened == False) and (now_rsi < 30) or ((now_MACD > now_MACD_signal) and (now_MACD >1)) or (now_close < now_200EMA):
      pos_opened = True
      open_price = now_close

    # closing a position
    if (pos_opened == True) and (now_rsi > 65) or ((now_MACD < now_MACD_signal) and (now_MACD <1)):
      pos_opened = False
      close_price = now_close
      pnl = close_price - open_price
      pnl_list.append(pnl)

  total_profit = round(sum(pnl_list), 2)
  num_of_trade = round(len(pnl_list), 2)
  avg_pnl = round(total_profit/num_of_trade, 2)

  return num_of_trade, total_profit, avg_pnl

**Calculating Total PNL**

In [124]:
backtest_values = []
for ticker in df_list:
  try:
    result = backtest(ticker)
    backtest_values.append(list(result))

    backtest_dict = dict(zip(ticker_list, backtest_values))
    backtest_raw = pd.DataFrame(backtest_dict.items(), columns=['Ticker', 'Values'])
    backtest_result = pd.concat([backtest_raw['Ticker'], pd.DataFrame(backtest_raw['Values'].tolist())],axis = 1)
    backtest_result.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade']
    Full_PNL = backtest_result['Total PNL'].sum().round(2)
  except:
    print('No Trade')

In [125]:
from dateutil import parser
import datetime

def start_date_transform(date, days_to_adjust):
  DT = parser.parse(date)
  DT = DT + datetime.timedelta(days=days_to_adjust)
  dstr = DT.strftime('%Y-%m-%d')
  return dstr

In [136]:
measure_start = start_date_transform(start_date, 287)
print('Trade period:', measure_start, 'to', end_date)
print('Total No of Trades Executed:', (backtest_result['No of Trade'].sum()))
print('Total PNL: USD', Full_PNL)
backtest_result.sort_values(by=['Total PNL', 'Avg PNL/ Trade'], ascending=False)

Trade period: 2022-01-01 to 2022-11-10
Total No of Trades Executed: 1708
Total PNL: USD 4110.94


,Ticker,No of Trade,Total PNL,Avg PNL/ Trade
2,O,118,1528.95,12.96
8,JEPI,100,618.65,6.19
0,ARCC,95,610.28,6.42
1,STAG,123,563.04,4.58
7,DIVO,108,464.14,4.30
12,QYLD,116,160.87,1.39
4,MAIN,122,83.25,0.68
11,PFLT,102,44.96,0.44
9,LTC,111,22.67,0.20
6,ACRE,99,21.64,0.22


In [128]:
# show all dataframe names in session
# https://stackoverflow.com/questions/44835358/pandas-list-of-dataframe-names
all_df_in_mem = %who_ls DataFrame
type(all_df_in_mem)

list